In [67]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
# Pkg.add("PyPlot")
# Pkg.add("PyCall")

In [68]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random
using Plots
# using PyPlot
# using PyCall

In [69]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [70]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

alpha = 0.9
#make beta a list of values
betas = [0.1, 0.2, 0.5, 0.7, 0.9, 0.95, 0.99, 0.999]
objectiv_values = []
exp_profits = []
CVaR_values = []

Any[]

In [71]:
for beta in betas
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)
    unregister(model, :p_DA)
    #reset the model
    

    # Define the decision variables for hour
    @variable(model, p_DA[1:hours])
    @variable(model, delta[1:W,1:hours])
    @variable(model, zeta)
    @variable(model, eta[1:W])

    # Define the objective function
    @objective(model, Max, (1-beta) * sum(1/W *
    (scenarios[i][hour,"price"] * p_DA[hour]
    + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)
    + beta  * (zeta - (1/(1-alpha)) * sum(1/W * eta[i] for i in 1:W)))

    # Define the constraints
    @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
    @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
    @constraint(model, [i in 1:W], eta[i] >= 0)
    @constraint(model, [i in 1:W], -1 * sum((scenarios[i][hour,"price"] * p_DA[hour]
    + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours) + zeta - eta[i] <= 0)


    # Solve the optimization problem
    optimize!(model)

    exp_profit = sum(1/W .*
    (scenarios[i][hour,"price"] * value.(p_DA[hour])
    + scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)

    CVaR = (value.(zeta) - (1/(1-alpha)) * sum(1/W .* value.(eta[i]) for i in 1:W))

    # Print the termination status
    status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Optimal solution found")

        println("p_DA: ", value.(p_DA))
        
        # RETURN OBJECTIVE value
        #println("Objective value: ", objective_value(model))
        #println("Expected profit: ", exp_profit)
        #println("CVaR: ", CVaR)
        push!(objectiv_values, objective_value(model))
        push!(exp_profits, exp_profit)
        push!(CVaR_values, CVaR)
    else
        println("No optimal solution found")
    end
end

Optimal solution found
p_DA: [200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, -2.2737367544323206e-13, 200.0, 0.0, 0.0, 0.0, 200.0]
Optimal solution found
p_DA: [200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, -2.2737367544323206e-13, 200.0, 0.0, 0.0, 0.0, 200.0]
Optimal solution found
p_DA: [200.0, 0.0, 0.0, 44.394115496773416, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 0.0, 200.0]
Optimal solution found
p_DA: [200.00000000000003, 0.0, 5.684341886080802e-14, 101.36268623281507, 7.283240855038468e-13, 200.00000000000117, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, -1.1084466677857563e-12, 200.0, 200.0, 0.0, 200.0, 0.0, 200.00000000000003, 0.0, 200.0]
Optimal solution found
p_DA: [200.0, 0.0, 2.842170943040401e-14, 101.36268623281752, 3.0730973321624333e-12, 200.00000000000327, 200.

In [72]:
#make a dataframe with the results
results = DataFrame(betas = betas, objective_values = objectiv_values, exp_profits = exp_profits, CVaR_values = CVaR_values)

#save the results
CSV.write("1_3_results/one_price_results.csv", results)

"1_3_results/one_price_results.csv"